In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import os, re
from os import listdir
import random

In [ ]:
# 셀레늄으로 크롬 브라우저를 연다.
# driver = webdriver.Chrome()
driver = webdriver.Chrome('chromedriver')

# '교육' 탭에 접속한 후, 로딩될 때까지 2초 기다린다.
driver.get('https://www.open.go.kr/search/theme/theme.do?themecd=00085')
time.sleep(2)

In [ ]:
# total number of documents
total_document = int(input("전체 문서 개수: " ))
print("전체 문서 개수: [%d]개" % (total_document))

# total number of pages
total_pages = total_document//10
print("전체 페이지 수: [%d]페이지" % (total_pages))

delay = 10.0
print("다운로드 딜레이: [%.1f] seconds" % (delay))
print("\n")


base_dir = os.path.dirname(os.path.abspath("__file__"))
count = 0
for page in range(1, total_pages+1):
    start_time = time.time()
    
    if page == 1:
        print("### 현재 페이지: [%d] 페이지" % (page))
    else:
        time.sleep(2.0)

        driver.execute_script("javascript:getResult('%d' , 'WOMN' )" % (page))
        print("\n\n### 현재 페이지: [%d] 페이지" % (page))
    
    # 현재 웹 페이지의 내용을 긁어서 원문 파일 목록에서 필요한 함수 인자를 가져온다.
    time.sleep(12.0)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    prdnDt_list = []
    prdnNstRgstNo_list= []
    for tr in soup.select("#result_false tr"):
        post_script = tr.td.a['href'][11:]
        tmp = re.findall("[']([0-9]+)['], [']([.]+)[']", post_script, re.IGNORECASE)[0]
        prdnDt_list.append(tmp[0]) 
        prdnNstRgstNo_list.append(tmp[1])
        
    if prdnDt_list != False:
#         print("함수 인자 가져오기 성공:", prdnDt_list[0], "-", prdnDt_list[-1])
        print ("함수 인자 [%d]개 가져오기 성공" % len(prdnDt_list))

    
    for prdnDt, prdnNstRgstNo in zip(prdnDt_list, prdnNstRgstNo_list):
        count += 1
 
        # 현재 웹브라우저에서 새 탭을 열고 이동한다.  
        driver.execute_script("window.open('')")
        driver.switch_to_window(driver.window_handles[1])

        # 원문 파일 다운로드 페이지에 접속한다.
        # driver.get(f'https://www.open.go.kr/pa/infoWonmun/cateSearch/wonmunOrginlDetail.do?prdnDt={prdnDt}&prdnNstRgstNo={prdnNstRgstNo}')
        download_url = "https://www.open.go.kr/pa/infoWonmun/cateSearch/wonmunOrginlDetail.do?prdnDt={0}&prdnNstRgstNo={1}".format(prdnDt,prdnNstRgstNo)
        driver.get(download_url)

        # 다운로드할 파일의 제목을 가져온다.
        file_soup = BeautifulSoup(driver.page_source, 'lxml')
        
        try:
            file_name = file_soup.select('span.fileName')[0].text
        except:
            print("%d) failure: 파일 이름을 찾을 수 없습니다." % (count))
            driver.execute_script("window.close()")
            driver.switch_to_window(driver.window_handles[0])
            continue

        # 파일을 다운로드 할 때 필요한 javascript 인자를 가져와서 함수를 실행한다.
        download_script = file_soup.select("p.fr")[0].a['onclick']

        # 단, 원본이 한글 파일이면 pdf로 변환하지 않는다.
#         if ".hwp" in download_script:
#             download_script = download_script.replace("'Y'","'N'")
        driver.execute_script(download_script)
            
        time.sleep(delay)
        try:
            driver.switch_to_alert().accept();
            print("%d) failure: [%s]" % (count, file_name))
        except:
            print("%d) success: [%s]" % (count, file_name))

        # 결재문서본문 이름을 가진 파일의 이름을 random 숫자를 넣어 바꾼다.
        for file in listdir(base_dir):
            if file.startswith("결재문서본문"):
                if not 'crdownload' in file:
                    og_file = os.path.join(os.path.dirname(os.path.abspath(file)), file)
                    os.rename(og_file, str(random.randint(1,100000)) + "결재문서본문" + str(random.randint(1,100000)) + ".pdf" )

        # 다운로드가 완료되었으니 새 탭을 닫고, 목록이 있는 탭으로 이동한다.
        driver.execute_script("window.close()")
        driver.switch_to_window(driver.window_handles[0])
    print("Elapsed time: %.2f sec" %(time.time() - start_time))